This notebook will be used as a base and support in the implementation of the nlpaug library for text augmentation.


Inspiration taken from : https://www.kaggle.com/code/andypenrose/text-augmentation-with-nlpaug

In [1]:
%load_ext autoreload
%autoreload 2

## Dataset

In [2]:
import pandas as pd

df = pd.read_csv("../../data/silver/df_cleantext_v0.csv")
df

,Category,Message
0,0,go until jurong point crazy available only in ...
1,0,ok lar joking wif u oni
2,1,free entry in number a wkly comp to win fa cup...
3,0,u dun say so early hor u c already then say
4,0,nah i dont think he goes to usf he lives aroun...
...,...,...
5152,1,this is the numbernd time we have tried number...
5153,0,will you b going to esplanade fr home
5154,0,pity was in mood for that soany other suggestions
5155,0,the guy did some bitching but i acted like id ...


In [3]:
import os
import sys
sys.path.append(os.path.abspath("../utils"))
from experiments_utils import print_and_highlight_diff

## Chunk for analysis

In [4]:
pd.set_option('display.max_colwidth', None)

sample = df['Message'].iloc[:5].astype(str).to_list()


In [5]:
import nlpaug.augmenter.char as nac
import nlpaug.augmenter.word as naw
import nlpaug.augmenter.word.context_word_embs as nawcwe
import nlpaug.augmenter.word.word_embs as nawwe
import nlpaug.augmenter.word.spelling as naws

## KeyboardAug method

In [6]:
aug = nac.KeyboardAug()
augmented_texts = aug.augment(sample)

print_and_highlight_diff(sample, augmented_texts)

--------------------------------------------------
Original: 20
go until jurong point crazy available only in bugis n great world la e buffet cine there got amore wat
Augmented: 20
go uGtiK jurong point craAt avai<QblF only in butiD n freaf world la e bIff3t cine there got amore wat 
--------------------------------------------------
Original: 6
ok lar joking wif u oni
Augmented: 6
ok lar uoOing wif u oni 
--------------------------------------------------
Original: 28
free entry in number a wkly comp to win fa cup final tkts numberst may number text fa to number to receive entry questionstd txt ratetcs apply numberovernumbers
Augmented: 28
CGee emtrt in number a wllH coNl to win fa cup BinQl tkts numberst may n&mbFr tDxF fa to number to receive entry quSstioGcGd txt ratetcs apply M&mbeDoverjumbefw 
--------------------------------------------------
Original: 11
u dun say so early hor u c already then say
Augmented: 11
u dun say so Warlg hor u c XlrewdJ $h2n say 
----------------------

Notes: 
1. The original criteria decided to not focus on special chars nor numbers so the cleaning acted accordingly. These two params should be changed. Same for upper/lower chars. 
2. Many characters of each modified word are changed and the result doesn't look realistic imo. This augmenter technique creates unrealistic results.


In [7]:
aug = nac.KeyboardAug(aug_word_p=0.1, include_numeric=False, include_special_char=False, include_upper_case=False)
augmented_texts = aug.augment(sample)

print_and_highlight_diff(sample, augmented_texts)

--------------------------------------------------
Original: 20
go until jurong point crazy available only in bugis n great world la e buffet cine there got amore wat
Augmented: 20
go until nugong point crazy available only in bugis n great world la e giffet cine there got amore wat 
--------------------------------------------------
Original: 6
ok lar joking wif u oni
Augmented: 6
ok lar joukng wif u oni 
--------------------------------------------------
Original: 28
free entry in number a wkly comp to win fa cup final tkts numberst may number text fa to number to receive entry questionstd txt ratetcs apply numberovernumbers
Augmented: 28
ffde entry in number a wkly comp to win fa cup final tkts numberst may number text fa to numvsr to receive entry questionstd txt ratetcs apply numnwrkvernumndfs 
--------------------------------------------------
Original: 11
u dun say so early hor u c already then say
Augmented: 11
u dun say so fagly hor u c already tusn say 
----------------------

Notes: 

The misspellings are very hardcore and artificial. They don't make sense to me. 


In [8]:
# help(nac.KeyboardAug)
# help(aug.augment)

## SpellingAug method

This method substitutes word by spelling mistake words dictionary

In [9]:
aug = naws.SpellingAug()
augmented_texts = aug.augment(sample)
print_and_highlight_diff(sample, augmented_texts)

--------------------------------------------------
Original: 20
go until jurong point crazy available only in bugis n great world la e buffet cine there got amore wat
Augmented: 20
go until jurong point craezy available only in bugis n grait world la and buffet cinema trere got amore want 
--------------------------------------------------
Original: 6
ok lar joking wif u oni
Augmented: 6
OK)][[... lar joking wife u oni 
--------------------------------------------------
Original: 28
free entry in number a wkly comp to win fa cup final tkts numberst may number text fa to number to receive entry questionstd txt ratetcs apply numberovernumbers
Augmented: 28
freer entry il lnumber al wkly comp too. wind fa coop final tkts numberst may numenber text fa tj number to receive entry questionstd txt ratetcs apply numberovernumbers 
--------------------------------------------------
Original: 11
u dun say so early hor u c already then say
Augmented: 11
u dun soy saw orly hor u c already then soy 

Notes: 

This method creates more realistic results than the previous technique.

In [10]:
aug = naws.SpellingAug(aug_p = 0.5)
augmented_texts = aug.augment(sample)
print_and_highlight_diff(sample, augmented_texts)

--------------------------------------------------
Original: 20
go until jurong point crazy available only in bugis n great world la e buffet cine there got amore wat
Augmented: 20
gou until jurong point carzy avalible ony in bugis in grate world la He buffet cinema there gotten amore eat 
--------------------------------------------------
Original: 6
ok lar joking wif u oni
Augmented: 6
Ok law joking wif You oni 
--------------------------------------------------
Original: 28
free entry in number a wkly comp to win fa cup final tkts numberst may number text fa to number to receive entry questionstd txt ratetcs apply numberovernumbers
Augmented: 28
free entry im nubmer a wkly comp ot winne fa cup final tkts numberst maybe namber's text fa te nummer ro receive entry questionstd txt ratetcs applying numberovernumbers 
--------------------------------------------------
Original: 11
u dun say so early hor u c already then say
Augmented: 11
You dun soy s early hor you c already thent stay 


Notes:

1. The results are way more realistic.
2. The dictionary includes upper case that should be treated later on.    


In [11]:
#help(naws.SpellingAug())
# help(aug.augment)

## SynonymAug

Substitute similar word according to WordNet/ PPDB synonym

Default is WordNet 

In [12]:
import nltk
nltk.download('averaged_perceptron_tagger_eng')

[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /home/maldu/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger_eng is already up-to-
[nltk_data]       date!


True

In [13]:
aug = naw.SynonymAug()
augmented_texts = aug.augment(sample)
print_and_highlight_diff(sample, augmented_texts)

--------------------------------------------------
Original: 20
go until jurong point crazy available only in bugis n great world la e buffet cine there got amore wat
Augmented: 21
go bad until jurong period weirdo available only in bugis n great world la e buffet cine there get amore wat 
--------------------------------------------------
Original: 6
ok lar joking wif u oni
Augmented: 6
ok lar joking wif u oni 
--------------------------------------------------
Original: 28
free entry in number a wkly comp to win fa cup final tkts numberst may number text fa to number to receive entry questionstd txt ratetcs apply numberovernumbers
Augmented: 29
free entrance in number a wkly comprehensive to win fa cup last tkts numberst may telephone number text fa to number to get entry questionstd txt ratetcs apply numberovernumbers 
--------------------------------------------------
Original: 11
u dun say so early hor u c already then say
Augmented: 12
u dun say thus early hor u c already and the

Notes:

1. Some add extra words and I don't understand why

- Original: 11
- u dun say so early hor u c already then say
- Augmented: 12
- u dun say so other hor u ampere second already then say 

2. Translations look more realistic to me 
3. Checking the params of the function it provides two databases of misspellings 'wordnet' and 'ppdb'.

In [14]:
aug = naw.SynonymAug(aug_p= 0.3)
augmented_texts = aug.augment(sample)
print_and_highlight_diff(sample, augmented_texts)

--------------------------------------------------
Original: 20
go until jurong point crazy available only in bugis n great world la e buffet cine there got amore wat
Augmented: 21
go until jurong full stop crazy available entirely in bugis n great reality lanthanum e buffet cine there got amore wat 
--------------------------------------------------
Original: 6
ok lar joking wif u oni
Augmented: 7
all right lar joking wif u oni 
--------------------------------------------------
Original: 28
free entry in number a wkly comp to win fa cup final tkts numberst may number text fa to number to receive entry questionstd txt ratetcs apply numberovernumbers
Augmented: 29
gratuitous entry in number a wkly comp to win fa cup final tkts numberst english hawthorn number text fa to act to receive entry questionstd txt ratetcs apply numberovernumbers 
--------------------------------------------------
Original: 11
u dun say so early hor u c already then say
Augmented: 12
u dun say indeed early hor 

Notes: 

1. Indeed we should be careful with the perc of augmented texts because it adds new words changing the length of the sentence. 
2. Some synonims are not fitting the meaning of the sentence imo.

This cannot be used unless we find a way to force 1-to-1 swapping...not even then. We should also restrict the type of synonym to swap to absolute synonyms :\

In [15]:
# help(naw.SynonymAug())

## WordEmbeddingAug method

This method inserts word randomly by word embeddings similarity. 

:param str model_type: Model type of word embeddings. Expected values include 'word2vec', 'glove' and 'fasttext'.

### Download models

The models used are coming from gensim library due to lack of local resources.


In [ ]:
from nlpaug.util.file.download import DownloadUtil

DownloadUtil.download_word2vec(dest_dir='./models')
DownloadUtil.download_glove('glove.6B', './models')
DownloadUtil.download_fasttext('wiki-news-300d-1M', './models')


### word2vec

Too heavy for my machine, leaving it for Github Actions running

In [17]:

aug = naw.WordEmbsAug(
    model_type='word2vec', model_path='./models/GoogleNews-vectors-negative300.bin',
    action="substitute")
augmented_texts = aug.augment(sample)
print_and_highlight_diff(sample, augmented_texts)


2025-01-03 17:02:05,707 - INFO - loading projection weights from ./models/GoogleNews-vectors-negative300.bin


FileNotFoundError: [Errno 2] No such file or directory: './models/GoogleNews-vectors-negative300.bin'

In [ ]:
aug = naw.WordEmbsAug(
    model_type='word2vec', model_path='./models/GoogleNews-vectors-negative300.bin',
    action="substitute", aug_p= 0.3)
augmented_texts = aug.augment(sample)
print_and_highlight_diff(sample, augmented_texts)

In [ ]:
aug = naw.WordEmbsAug(
    model_type='word2vec', model_path='./models/GoogleNews-vectors-negative300.bin',
    action="substitute", aug_p= 0.8)
augmented_texts = aug.augment(sample)
print_and_highlight_diff(sample, augmented_texts)

### glove 6B 100d

In [ ]:

aug = naw.WordEmbsAug(
    model_type='glove', model_path='./models/glove.6B.100d.txt',
    action="substitute")
augmented_texts = aug.augment(sample)
print_and_highlight_diff(sample, augmented_texts)

In [ ]:

aug = naw.WordEmbsAug(
    model_type='glove', model_path='./models/glove.6B.100d.txt',
    action="substitute",  aug_p= 0.3)
augmented_texts = aug.augment(sample)
print_and_highlight_diff(sample, augmented_texts)

In [ ]:

aug = naw.WordEmbsAug(
    model_type='glove', model_path='./models/glove.6B.100d.txt',
    action="substitute",  aug_p= 0.8)
augmented_texts = aug.augment(sample)
print_and_highlight_diff(sample, augmented_texts)

### fasttext

In [ ]:
aug = naw.WordEmbsAug(
    model_type='fasttext', model_path='./models/wiki-new-300d-1M.vec',
    action="substitute")
augmented_texts = aug.augment(sample)
print_and_highlight_diff(sample, augmented_texts)

In [ ]:

aug = naw.WordEmbsAug(
    model_type='fasttext', model_path='./models/wiki-new-300d-1M.vec',
    action="substitute", aug_p= 0.3)
augmented_texts = aug.augment(sample)
print_and_highlight_diff(sample, augmented_texts)

In [ ]:
aug = naw.WordEmbsAug(
    model_type='word2vec', model='nlpaug.model.word_embs.nmw.Word2vec()',
    action="substitute", aug_p= 0.8)
augmented_texts = aug.augment(sample)
print_and_highlight_diff(sample, augmented_texts)

In [41]:
# import nlpaug.augmenter.word as naw
# aug = naw.WordEmbsAug(model_type='word2vec', model='nlpaug.model.word_embs.nmw.Word2vec()',action="substitute", aug_p= 0.8)
# augmented_texts = aug.augment(sample)
# print_and_highlight_diff(sample, augmented_texts)

AttributeError: 'str' object has no attribute 'get_vocab'

In [ ]:
# help(naw.WordEmbsAug)

In [ ]:
aug = naw.WordEmbsAug(
    model_type='fasttext', model_path='./models/wiki-new-300d-1M.vec',
    action="substitute", aug_p= 0.8)
aaugmented_texts = aug.augment(sample)
print_and_highlight_diff(sample, augmented_texts)

## ContextualWordEmbs method

I'll try SqueezeBERT since the resources are limited 

In [ ]:
aug = naw.ContextualWordEmbsAug(
    model_path='distilbert-base-uncased', action="substitute", aug_p= 0.3)
augmented_text = aug.augment(sample)
print_and_highlight_diff(sample, augmented_texts)

In [31]:
aug = nawcwe.ContextualWordEmbsAug(
    model_path=model_dir, 
    action ='substitute', 
    aug_p= 0.8)
augmented_texts = aug.augment(sample)
print_and_highlight_diff(sample, augmented_texts)

--------------------------------------------------
Original: 1
Y
Augmented: 5
my sample just goes here 


In [ ]:
# help(nawcwe.ContextualWordEmbsAug)